In [83]:
# This serves as a template which will guide you through the implementation of this task.  It is advised
# to first read the whole template and get a sense of the overall structure of the code
# First, we import necessary libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic, ExpSineSquared
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [84]:
def data_loading():
    """
    This function loads the training and test data, preprocesses it, removes the NaN values and interpolates the missing
    data using imputation

    Parameters
    ----------
    Returns
    ----------
    X_train: matrix of floats, training input with features
    y_train: array of floats, training output with labels
    X_test: matrix of floats: dim = (100, ?), test input with features
    """
    # Load training data
    train_df = pd.read_csv("/home/otps3141/Documents/Git Hub/IML-2023/P2/train.csv")



    # Load test data
    test_df = pd.read_csv("/home/otps3141/Documents/Git Hub/IML-2023/P2/test.csv")



    # Perform data preprocessing, imputation and extract X_train, y_train and X_test using mean values

    # Use interpolation for missing values. Interpolate cannot handle missing starting or end values. So fill these up with mean()

    # Why does this not work with RBF?

    ### Training set

    new_train_df = train_df.interpolate(method="akima")
    new_train_df = new_train_df.fillna(train_df.mean())


    # Encode season data

    binary_version_seasons = pd.get_dummies(new_train_df['season'])
    new_train_df = new_train_df.drop(columns=['season']).join(binary_version_seasons)

    ### Test set

    new_test_df = test_df.interpolate(method="akima")
    new_test_df = new_test_df.fillna(new_test_df.mean())

    # Encode season data

    new_test_df = new_test_df.drop(columns=['season']).join(binary_version_seasons)

    ###

    seasons = train_df['season'].unique().tolist()

    season_X_train = {}
    season_y_train = {}
    season_X_test = {}

    for season in seasons:

        season_X_train[f'{season}'] = new_train_df[new_train_df[f'{season}'] == 1].drop(columns=["price_CHF"])
        season_y_train[f'{season}'] = new_train_df[new_train_df[f'{season}'] == 1]['price_CHF']
        season_X_test[f'{season}'] = new_test_df[new_test_df[f'{season}'] == 1]



    # assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (
    #             X_test.shape[0] == 100), "Invalid data shape"

    return season_X_train, season_y_train, season_X_test, seasons


In [85]:
train_df = pd.read_csv("/home/otps3141/Documents/Git Hub/IML-2023/P2/train.csv")

test_df = pd.read_csv("/home/otps3141/Documents/Git Hub/IML-2023/P2/test.csv")

# seasonal_data_indexes = pd.DataFrame(test_df['season'])

# seasonal_data_indexes

# seasons = train_df['season'].unique().tolist()

# season_split_data = {}
# y_values_seasons = {}
# test_season_split_data = {'seasonal_data': seasonal_data_indexes}

# # Impute and split the data over the seasons.
# temp = train_df.loc[train_df['season'] == 'winter']
# temp_interpolid = temp.interpolate('akima').fillna(temp.mean())



# # print(temp)
# print(temp_interpolid)
#     # season_split_data[season] = temp_interpolid.drop(columns=['price_CHF'], axis=1)
#     # y_values_seasons[season] = temp_interpolid['price_CHF']

#     # temp_test = test_df.loc[test_df['season'] == season]
#     # temp_test_interpolid = temp_test.interpolate('akima').fillna(temp_test.mean())
#     # test_season_split_data[season] = temp_test_interpolid



In [86]:
new_train_df = train_df.interpolate(method="akima")
new_train_df = new_train_df.fillna(train_df.mean())


# Encode season data

binary_version_seasons = pd.get_dummies(new_train_df['season'])
new_train_df = new_train_df.drop(columns=['season']).join(binary_version_seasons)

### Test set

new_test_df = test_df.interpolate(method="akima")
new_test_df = new_test_df.fillna(new_test_df.mean())

# Encode season data

new_test_df = new_test_df.drop(columns=['season']).join(binary_version_seasons)

seasons = train_df['season'].unique().tolist()

season_X_train = {}
season_y_train = {}
season_X_test = {}

for season in seasons:

    season_X_train[f'{season}'] = new_train_df[new_train_df[f'{season}'] == 1].drop(columns=["price_CHF"])
    season_y_train[f'{season}'] = new_train_df[new_train_df[f'{season}'] == 1]['price_CHF']
    season_X_test[f'{season}'] = new_test_df[new_test_df[f'{season}'] == 1]



/tmp/ipykernel_574927/64763305.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_train_df = new_train_df.fillna(train_df.mean())
/tmp/ipykernel_574927/64763305.py:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_test_df = new_test_df.fillna(new_test_df.mean())


In [88]:
def calculate_R2(y_pred, y):

    R2 = r2_score(y, y_pred)

    assert np.isscalar(R2)
    return R2


def calculate_RMSE(y_pred, y):
    """This function takes test data points (X and y), and computes the empirical RMSE of
    predicting y from X using a linear model with weights w.

    Parameters
    ----------
    y_pred: array of floats
    y: array of floats, dim = (15,), input labels

    Returns
    ----------
    RMSE: float: dim = 1, RMSE value
    """

    # Determine mean squared error
    RMSE = mean_squared_error(y, y_pred) ** 0.5

    assert np.isscalar(RMSE)
    return RMSE


In [93]:
def modeling_and_prediction(X_train, y_train, X_test, seasons):
    """
    This function defines the model, fits training data and then does the prediction with the test data 

    Parameters
    ----------
    X_train: matrix of floats, training input with 10 features
    y_train: array of floats, training output
    X_test: matrix of floats: dim = (100, ?), test input with 10 features

    Returns
    ----------
    y_test: array of floats: dim = (100,), predictions on test set
    """

    y_pred=np.zeros(X_test['spring'].shape[0])
    # y_pred = np.zeros(X_test['seasonal_data'].shape[0])
    #TODO: Define the model and fit it using training data. Then, use test data to make predictions

    # Define n_fold cross validation
    kf = KFold(10)

    # Define error
    error_model = 'R2'

    # Define model
    model = GaussianProcessRegressor(kernel=DotProduct())


    # Define storage for y prediction
    y_pred_dict = {}

    for season in seasons:
        X_test_season = X_test[season].drop(columns=seasons)
        X_train_season = X_train[season].drop(columns=seasons)
        y_train_season = y_train[season]

        y_pred_dict[season] = _fit_kfold(kf_splitter=kf, X_test=X_test_season, X_train=X_train_season,
                                    y_train=y_train_season, error_method=error_model, model=model)

        # indices = seasonal_data_indexes.index[seasonal_data_indexes['season'] == season].to_numpy()
        # y_pred[indices] = y_pred_dict[season]

    y_pred = y_pred_dict['spring']


    assert y_pred.shape == (100,), "Invalid data shape"
    return y_pred

In [90]:
# def modeling_and_prediction(X_train, y_train, X_test):
#     """
#     This function defines the model, fits training data and then does the prediction with the test data

#     Parameters
#     ----------
#     X_train: matrix of floats, training input with 10 features
#     y_train: array of floats, training output
#     X_test: matrix of floats: dim = (100, ?), test input with 10 features

#     Returns
#     ----------
#     y_test: array of floats: dim = (100,), predictions on test set
#     """

#     # Create storage array for predicted y values
#     y_pred1 = np.zeros(X_test.shape[0])

#     # Leave out seasons data
#     # X_train = X_train[:, 1:]

#     # Define final test data
#     X_test1 = X_test

#     X_train = np.array(X_train)
#     y_train = np.array(y_train)


#     # Storage array for RMSE values
#     RMSE_mat = np.zeros(9)
#     R2_mat = np.zeros(9)

#     # Define n_fold cross validation
#     kf = KFold(9)
#     mat_i = 0

#     # Storage for all n_fold models
#     models = []

#     # Perform Cross validation
#     for train_index, test_index in kf.split(X_train):
#         X_train_folds = X_train[train_index]
#         y_train_folds = y_train[train_index]

#         gpr = GaussianProcessRegressor(kernel=DotProduct())
#         gpr.fit(X_train_folds, y_train_folds)

#         X_test = X_train[test_index]
#         y_test = y_train[test_index]

#         y_pred, sigma = gpr.predict(X_test, return_std=True)

#         # print(sigma)

#         models.append(gpr)

#         RMSE_mat[mat_i] = calculate_RMSE(y_pred, y_test)
#         R2_mat[mat_i] = calculate_R2(y_pred, y_test)

#         mat_i = mat_i + 1

#     # print(RMSE_mat)
#     # print(R2_mat)

#     # Determine best model
#     best_index_RMSE = np.argmin(RMSE_mat)
#     best_index_R2 = np.argmin(R2_mat)

#     final_gpr_RMSE = models[best_index_RMSE]
#     final_gpr_R2 = models[best_index_R2]

#     # y_pred = final_gpr_RMSE.predict(X_test1[:, 1:])
#     y_pred = final_gpr_R2.predict(X_test1[:, :])


#     # print(gpr.score(X_train[:,1:], y_train))

#     '''
#     ps = PolynomialCountSketch(degree=3, random_state=1)
#     X_features = ps.fit_transform(X)
#     #print(np.exp(X_train[5,1]))
#     '''
#     # plt.plot(np.exp(X_train[:,1]),np.exp(y_train),'.')

#     assert y_pred.shape == (100,), "Invalid data shape"
#     return y_pred

In [91]:
def _fit_kfold(kf_splitter, X_test, X_train, y_train, model, error_method):
    """"
    Helper function to compute the KFolds and keep the rest of the modeling function cleaner.
    The model is the machine learning model that fits the data.
    """
    # Storage for errors
    error_matrix = np.zeros(kf_splitter.get_n_splits())

    # Storage for all n_fold models
    models = []

    # Perform Cross validation
    for i, (train_index, test_index) in enumerate(kf_splitter.split(X_train)):
        X_train_folds = X_train[train_index]
        y_train_folds = y_train[train_index]

        X_test_folds = X_train[test_index]
        y_test_folds = y_train[test_index]

        # The model is refitted every time.
        model.fit(X_train_folds, y_train_folds)
        y_pred_folds, sigma = model.predict(X_test_folds, return_std=True)

        models.append(model)

        if error_method == 'RMSE':
            error_matrix[i] = calculate_RMSE(y_pred_folds, y_test_folds)

        elif error_method == 'R2':
            error_matrix[i] = calculate_R2(y_pred_folds, y_test_folds)

        else:
            raise NotImplemented

    best_index = np.argmin(error_matrix)
    best_fit = models[best_index]

    y_pred = best_fit.predict(X_test)

    return y_pred


In [94]:
# Data load
X_train, y_train, X_test, seasons = data_loading()
# The function retrieving optimal LR parameters
y_pred = modeling_and_prediction(X_train, y_train, X_test, seasons)
# Save results in the required format

dt = pd.DataFrame(y_pred)

dt.columns = ['price_CHF']
dt.to_csv('/home/otps3141/Documents/Git Hub/IML-2023/P2/results.csv', index=False)
print("\nResults file successfully generated!")

/tmp/ipykernel_574927/2588899548.py:33: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_train_df = new_train_df.fillna(train_df.mean())
/tmp/ipykernel_574927/2588899548.py:44: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  new_test_df = new_test_df.fillna(new_test_df.mean())


KeyError: "None of [Int64Index([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,\n            ...\n            215, 216, 217, 218, 219, 220, 221, 222, 223, 224],\n           dtype='int64', length=202)] are in the [columns]"